# **Computational Physics Project**

### Numerical Integration of Quantum Probability

#### **Preamble**
First, we import the necessary modules for the project and define any constants we expect to use (such as machine epsilon).

In [1]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

%matplotlib inline

# Define machine epsilon for default floats
eps = np.finfo(float).eps

#### **Newton-Cotes Integration**
We will use object-oriented programming for this project. Let us define a generic integration class which we will use for argument validation. This will be a parent class for the `TrapeziumRule` and `SimpsonsRule` classes which will inherit its properties. We define the required attributes for the Newton-Cotes methods in this class.

In [ ]:
class NC_Integration(object):
    """Generic class to hold attributes for Newton-Cotes integration methods.
    
    This class does not contain any processing methods of its own. It is used
    to hold all of the variables required to perform a Newton-Cotes
    integration. The class will provide argument validation and processing.
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the integral.

    b : scalar
        The upper limit of the integral.

    max_error : scalar
        The accuracy goal for the integration.

    max_steps : scalar
        The maximum number of steps to perform the integral.
    
    plot : boolean
        A boolean to indicate if plot should be output.

    integral : scalar
        The value of the integral.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
    
    steps : integer
        The number of recursions until desired accuracy achieved.
    
    """
    
    def __init__(self, fn, a, b, max_error, max_steps, plot):
        """Initialises NC_Integration class.
        
        Declares the required attributes and performs necessary argument
        validation for Newton-Cotes integration methods.
        
        Parameters
        ----------
        fn : function
            The integrand.
        
        a : scalar
            The lower limit of the integral.
        
        b : scalar
            The upper limit of the integral.
        
        max_error : scalar
            The accuracy goal for the integration.
        
        max_steps : integer
            The maximum number of steps allowed to achieve accuracy goal.
        
        plot : boolean
            A boolean to indicate if plot should be output.
        
        Raises
        ------
        ValueError
            If `fn` is not a callable function.
            If `a`, `b` are not scalars or `a` is not less than `b`.
            If `max_error` is not a scalar or `max_error` is too small.
            If `max_steps` is not an integer or is too small.
        
        """
        
        if callable(fn) == False:
            raise ValueError("The integrand `fn` must be a callable function.")
        
        if np.isfinite(a) == False or np.isfinite(b) == False:
            raise ValueError("Integral limits `a` and `b` must be finite " +
                             "numbers.")
        
        if b < a:
            raise ValueError("The upper limit `b` must be larger than the " +
                             "lower limit `a`.")
        
        if max_error < 1.0e3 * eps:
            raise ValueError("The specified accuracy `max_error` is too small.")
        
        if max_steps < 2:
            raise ValueError("Argument `max_steps` must be an integer " +
                             "greater than 2.")
        
        # Validation successful, initiate class variables
        self.fn = fn
        self.a = a
        self.b = b
        self.max_error = max_error
        self.max_steps = int(max_steps)
        self.plot = plot
        
        self.integral = 0.0
        self.error = 1.0e30
        self.iterations = 0
        self.steps = 0

#### **Trapezium Rule**
Let us define a trapezium integration class which can be created with a function and some integration region. The class will contain information about the integral, including the integral value, the error in the value and the number of iterations executed.

In [ ]:
class TrapeziumRule(NC_Integration):
    """Class to perform integration using the trapezium rule.
    
    This class uses an iterative approach to estimate an integral using the
    trapezium rule. We iteratively increase the number of function evaluations
    until the desired accuracy is achieved. The class inherits from the generic
    NC_Integration class and inherits all of its attributes.
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the integral.

    b : scalar
        The upper limit of the integral.

    max_error : scalar, optional, default = 1.0e-3
        The accuracy goal for the integration.

    max_steps : scalar, optional, default = 25
        The maximum number of steps to perform the integral.
    
    plot : boolean
            A boolean to indicate if plot should be output.

    integral : scalar
        The value of the integral.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
    
    """
    
    def __init__(self, fn, a, b, max_error=1.0e-3, max_steps=25, plot=True):
        """Initialises TrapeziumRule class.
        
        This class performs trapezium rule integration iteratively until the
        desired accuracy is achieved. The class inherits its attributes from the
        NC_Integration parent class, which performs all of the necessary
        argument validation.
        
        Parameters
        ----------
        fn : function
            The integrand.
        
        a : scalar
            The lower limit of the integral.
        
        b : scalar
            The upper limit of the integral.
        
        max_error : scalar, optional, default = 1.0e-3
            The accuracy goal for the integration.
        
        max_steps : integer, optional, default = 25
            The maximum number of steps to perform the integral.
        
        plot : boolean, optional, default = True
            A boolean to indicate if plot should be output.
        
        """
        
        # Inherit from NC_Integration class
        super().__init__(fn, a, b, max_error=max_error, max_steps=max_steps, plot=plot)
        
        # Compute the integral
        self.integrate()
    
    def integrate(self):
        """Performs the integration using the trapezium rule.
        
        This method makes calls to the trapezium step method to iteratively
        update the estimate of the integral. The function does not return any
        value - it updates the class attributes when the integration is
        complete.
        
        Raises
        ------
        ValueError
            If the integral did not converge to the desired accuracy in the
                given number of steps.
        
        See Also
        --------
        SimpsonsRule.integrate : performs the integral using Simpson's Rule.
        MC_Integration.integrate : perfomrs integral using Monte Carlo methods.

        Notes
        -----
        This function uses an iterative approach to repeatedly apply the
        trapezium rule with a reduced step size until the desired accuracy is
        achieved. The additional function evaluations required for each
        iterative step are performed in the static method `trapezium_step`.

        References
        ----------
        Numerical Recipes in C, Integration of Functions, Elementary Algorithms
        W. H. Press, Cambridge University Press

        Examples
        --------
        Integrate a function f on an interval x = 0.0 to x = 1.5708

        >>> def f(x): return np.cos(x)
        >>> t = TrapeziumRule(f, 0.0, 1.5708)
        >>> t.integral
        0.9997991933740931

        Specify an accuracy goal of 1.0e-6 for the integral

        >>> t = TrapeziumRule(f, 0.0, 1.5708, max_error=1.0e-6)
        >>> t.integral
        0.9999998039009074

        Get error estimate and number of function evaluations

        >>> t = TrapeziumRule(f, 0.0, 1.5708)
        >>> (t.integral, t.error, t.iterations)
        (0.9997991933740931, 0.0006025166533906168, 33)

        """
        
        # Initialise variable to hold integral value
        t0 = -1.0e30
        iters = 1
        
        # Repeat for max_steps iterations
        for j in range(1, self.max_steps):
            
            # Update integral estimate from previous iteration t0
            t1 = TrapeziumRule.trapezium_step(self.fn, self.a, self.b, j, t0)
            
            # At each step, we perform 2 ** (j-2) function calls, ceil ensures
            # that we have two total iterations for j = 1
            iters += np.ceil(2 ** (j - 2))
            
            # Prevent erroneous early convergence
            if j >= 2:
                
                # Exit if error restriction satisfied
                if (abs(t1 - t0) < self.max_error * abs(t1) or
                    (abs(t0) < eps and abs(t1) < eps)):
                    
                    # Update class attribute values
                    self.integral = t1
                    self.error = abs(t1 - t0) * abs(t1)
                    self.iterations = int(iters)
                    
                    if self.plot == True:
                        self.make_plot()
                    
                    # No further iterations required, return from function
                    return
            
            # Update integral estimate
            t0 = t1
        
        # Integral did not converge in max_steps, raise error
        raise ValueError("The integral did not converge in the maximum number " +
                         "of steps.")
    
    def make_plot(self):
        # Internal function for graph plotting

        # Plot the integrand
        X1 = np.linspace(self.a, self.b, 500)
        Y1 = [self.fn(x1) for x1 in X1]
        
        # Plot the abscissas
        X2 = np.linspace(self.a, self.b, self.iterations)
        Y2 = [self.fn(x2) for x2 in X2]
        
        plt.plot(X1, Y1, label="f", color="black")
        plt.plot(X2, Y2, "-x", ls="dashed", label="Trapezium Rule", color="red")
        plt.fill_between(X2, Y2, color="#eeeeee")
        plt.legend()
        plt.xlabel("$x$")
        plt.ylabel("$y$")
        plt.grid()
        plt.show()
    
    @staticmethod
    def trapezium_step(fn, a, b, n, s=0.0):
        """Computes the n-th step of the trapzeium rule integration.
        
        This function performs the n-th step of the trapezium rule using the
        value `s`, which is the integral estimate from the previous step. This
        is a static method to be re-used in the `SimpsonsRule` class.
        
        Parameters
        ----------
        fn : function
            The integrand.
        
        a : scalar
            The lower limit of the integral.
        
        b : scalar
            The upper limit of the integral.
        
        n : integer
            The step (iteration) of the integral.
        
        s : scalar, optional, default = 0.0
            Estimate of the integral from the previous iteration.
        
        Returns
        -------
        s1 : scalar
            Estimate of the integral from this iteration.
        
        See Also
        --------
        TrapeziumRule.integrate : performs integral using the trapezium rule.

        Notes
        -----
        The integral is performed using a iterative approach to the trapezium
        rule where each iterative step only evaluates the function at the new
        interval spacing without repeating evaluations. The result of the new
        evaluations is added to the integral from the previous iterative step,
        which is passed as an argument.

        References
        ----------
        Numerical Recipes in C, Integration of Functions, Elementary Algorithms
        W. H. Press, Cambridge University Press

        """
        
        # Error validation occurs in parent class, proceed with integration
        if n == 1:
            # First iterative step requires evaluation at endpoints
            return 0.5 * (b - a) * (fn(a) + fn(b))
        else:
            # Declare variable to hold sum of new evaluations
            sum_yn = 0.0
            
            # Declare interval increment for this iteration
            inc = (b - a) / (2 ** (n - 2))
            
            # Declare first new abscissa to evaluate
            xn = a + 0.5 * inc
            
            # Sample to upper abscissa limit, account for machine accuracy
            while xn < b - eps:
                sum_yn += fn(xn)
                xn += inc
            
            # Return the new integral estimate
            return 0.5 * (s + inc * sum_yn)

#### **Trapezium Rule - Validation**
We will perform some known integrals to verify that the trapezium rule routine is integrating correctly.

In [ ]:
def f(x):
    return x ** 2

t = TrapeziumRule(f, -1.0, 1.0, max_error=1.0e-4, plot=False)
print("Integral: ", t.integral)
print("Absolute Error: ", t.error)
print("Function Evaluations: ", t.iterations)
print("\n")
# Answer = 2 / 3

def f(x):
    return 1 - 2 * x ** 2

t = TrapeziumRule(f, 0.0, 1.0, max_error=1.0e-5, plot=False)
print("Integral: ", t.integral)
print("Absolute Error: ", t.error)
print("Function Evaluations: ", t.iterations)
print("\n")
# Answer = 1 / 3

def f(x):
    return np.cos(x)

t = TrapeziumRule(f, 0.0, np.pi, max_error=1.0e-3, plot=False)
print("Integral: ", t.integral)
print("Absolute Error: ", t.error)
print("Function Evaluations: ", t.iterations)
print("\n")
# Answer = 0

def f(x):
    return -np.exp(-(x - 0.5) ** 2)

t = TrapeziumRule(f, 0.0, 1.0, max_error=1.0e-6, plot=False)
print("Integral: ", t.integral)
print("Absolute Error: ", t.error)
print("Function Evaluations: ", t.iterations)
# Answer = -0.922562012826

In [ ]:
def f(x):
    return np.exp(-x ** 2) / np.sqrt(np.pi)

t = TrapeziumRule(f, 0.0, 2.0, max_error=1.0e-6, plot=True)
print("Integral: ", t.integral)
print("Absolute Error: ", t.error)
print("Function Evaluations: ", t.iterations)

#### **Simpson's Rule**
Let us define a Simpson's integration class which can be created with a function and some integration region. The class will contain information about the integral, including the integral value, the error in the value and the number of iterations executed.

In [ ]:
class SimpsonsRule(NC_Integration):
    """Class to perform integration using the Simpson's rule.
    
    This class uses an iterative approach to estimate an integral using the
    Simpson's rule. We iteratively increase the number of function evaluations
    until the desired accuracy is achieved. The class inherits its attributes
    from the generic NC_Integration class. This class uses the static
    `trapezium_step` method from the `TrapeziumRule` class to calculate an
    estimate for the Simpson's rule.
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the integral.

    b : scalar
        The upper limit of the integral.

    max_error : scalar, optional, default = 1.0e-3
        The accuracy goal for the integration.

    max_steps : scalar, optional, default = 25
        The maximum number of steps to perform the integral.
    
    plot : boolean
            A boolean to indicate if plot should be output.

    integral : scalar
        The value of the integral.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
    
    """
    def __init__(self, fn, a, b, max_error=1.0e-3, max_steps=25, plot=True):
        """Initialises SimpsonsRule class.
        
        This class performs numerical integration using the Simpson's rule. The
        class relies on the static trapezium_step function and the relationship
        between two consecutive iterative steps of the trapezium rule and the
        Simpson's rule to iteratively update the integral value until the
        desired accuracy is achieved. The class inherits its attributes from the
        NC_Integration parent class, which performs all of the necessary
        argument validation.
        
        Parameters
        ----------
        fn : function
            The integrand.
        
        a : scalar
            The lower limit of the integral.
        
        b : scalar
            The upper limit of the integral.
        
        max_error : scalar, optional, default = 1.0e-3
            The accuracy goal for the integration.
        
        max_steps : integer, optional, default = 25
            The maximum number of steps to perform the integral.
        
        plot : boolean, optional, default = True
            A boolean to indicate if plot should be output.
        """
        
        # Inherit from NC_Integration class
        super().__init__(fn, a, b, max_error=max_error, max_steps=max_steps, plot=plot)
        
        # Compute the integral
        self.integrate()
    
    def integrate(self):
        """Performs the integration using the Simpson's rule.
        
        This method makes calls to the trapezium step method to iteratively
        update the estimate of the integral. The Simpson's rule estimate is
        calculated using the relationship with the trapezium rule. The function
        does not return any value - it updates the class attributes when the
        integration is complete.
        
        Raises
        ------
        ValueError
            If the integral did not converge to the desired accuracy in the
                given number of steps.
        
        See Also
        --------
        TrapeziumRule.integrate : performs integral using the trapezium rule.
        TrapeziumRule.trapezium_step : performs one iteration of trapezium rule.
        MC_Integration.integrate : performs integral using Monte Carlo methods.

        Notes
        -----
        This function uses an iterative approach to repeatedly apply the
        trapezium rule with a reduced step size until the desired accuracy is
        achieved. The additional function evaluations required for each
        iterative step are performed in the static method `trapezium_step`. The
        function exploits the relationship between two iterative steps of the
        trapezium rule to estimate the integral using the Simpson's rule.

        References
        ----------
        Numerical Recipes in C, Integration of Functions, Elementary Algorithms
        W. H. Press, Cambridge University Press

        Examples
        --------
        Integrate a function f on an interval x = 0.0 to x = 1.5708

        >>> def f(x): return np.cos(x)
        >>> s = SimpsonRule(f, 0.0, 1.5708)
        >>> s.integral
        1.0000082955949947

        Specify an accuracy goal of 1.0e-6 for the integral

        >>> s = TrapeziumRule(f, 0.0, 1.5708, max_error=1.0e-6)
        >>> s.integral
        1.0000000322585565

        Get error estimate and number of function evaluations

        >>> s = TrapeziumRule(f, 0.0, 1.5708)
        >>> (s.integral, s.error, s.iterations)
        (1.0000082955949947, 0.00012629064304969795, 9)

        """
        
        # Declare variables for trapezium rule, Simpson's rule estimates
        t0 = -1.0e30
        s0 = -1.0e30
        iters = 1
        
        # Repeat for max_steps iterations
        for j in range(1, self.max_steps):
            
            # Update integral estimate from previous iteration t0
            t1 = TrapeziumRule.trapezium_step(self.fn, self.a, self.b, j, t0)
            
            # Calculate Simpson's rule estimate, S_i = (4T_(i+1) - T_i) / 3
            s1 = (4.0 * t1 - t0) / 3.0
            
            # At each step, we perform 2 ** (j-2) function calls, ceil ensures
            # that we have two total iterations for j = 1
            iters += np.ceil(2 ** (j - 2))
            
            # Prevent erroneous early convergence to solution
            if j >= 2:
                
                # Exit if error restriction satisfied
                if (abs(s1 - s0) < self.max_error * abs(s1) or
                    (abs(s0) < eps and abs(s1) < eps)):
                    
                    # Update class attributes
                    self.integral = s1
                    self.error = abs(s1 - s0) * abs(s1)
                    self.iterations = int(iters)
                    
                    if self.plot == True:
                        self.make_plot()
                    
                    # No further iterations required, return from function
                    return
            
            # Update trapezium rule and Simpson's rule estimates
            t0 = t1
            s0 = s1
        
        # Integral did not converge in max_steps, raise error
        raise ValueError("The integral did not converge in the maximum number" +
                         "of steps.")
    
    def make_plot(self):
        # Internal function for graph plotting

        # Plot the integrand
        X1 = np.linspace(self.a, self.b, 500)
        Y1 = [self.fn(x1) for x1 in X1]
        plt.plot(X1, Y1, label="f", color="black")
        
        # Plot the absciccas
        X2 = np.linspace(self.a, self.b, self.iterations)
        Y2 = [self.fn(x2) for x2 in X2]
        plt.plot(X2, Y2, "x", color="red")

        # Plot the Lagrange interpolation between each set of three points
        for i in range(1, len(X2) - 1, 2):

            # Extract three relevant abscissas
            X3 = X2[i - 1:i + 2]
            Y3 = Y2[i - 1:i + 2]

            # Apply second order Lagrange polynomial between three abscissas
            lp = interpolate.lagrange(X3, Y3)
            
            # Plot the interpolated polynomial over the three abscissas
            X4 = np.linspace(X2[i - 1], X2[i + 1], 5)
            Y4 = [lp(x) for x in X4]
            plt.plot(X4, Y4, ls="dashed", color="red")
            plt.fill_between(X4, Y4, color="#eeeeee")

        # Include interpolated Lagrange polynomial in the legend
        plt.plot(0, 0, ls="dashed", label="Simpson's Rule", color="red")
        
        plt.legend()
        plt.xlabel("$x$")
        plt.ylabel("$y$")
        plt.grid()
        plt.show()

#### **Simpson's Rule - Validation**
We will perform some known integrals to verify that Simpson's rule routine is integrating correctly.

In [ ]:
def f(x):
    return x ** 2

s = SimpsonsRule(f, -1.0, 1.0, max_error=1.0e-4, plot=False)
print("Integral: ", s.integral)
print("Absolute Error: ", s.error)
print("Function Evaluations: ", s.iterations)
print("\n")
# Answer = 2 / 3

def f(x):
    return 1 - 2 * x ** 2

s = SimpsonsRule(f, 0.0, 1.0, max_error=1.0e-5, plot=False)
print("Integral: ", s.integral)
print("Absolute Error: ", s.error)
print("Function Evaluations: ", s.iterations)
print("\n")
# Answer = 1 / 3

def f(x):
    return np.cos(x)

s = SimpsonsRule(f, 0.0, np.pi, max_error=1.0e-3, plot=False)
print("Integral: ", s.integral)
print("Absolute Error: ", s.error)
print("Function Evaluations: ", s.iterations)
print("\n")
# Answer = 0

def f(x):
    return -np.exp(-(x - 0.5) ** 2)

s = SimpsonsRule(f, 0.0, 1.0, max_error=1.0e-6, plot=False)
print("Integral: ", s.integral)
print("Absolute Error: ", s.error)
print("Function Evaluations: ", s.iterations)
# Answer = -0.922562012826

In [ ]:
def f(x):
    return np.exp(-x ** 2) / np.sqrt(np.pi)

s = SimpsonsRule(f, 0.0, 2.0, max_error=1.0e-6, plot=True)
print("Integral: ", s.integral)
print("Error: ", s.error)
print("Function Evaluations: ", s.iterations)

#### **Monte Carlo Integration**
We wish to use Monte Carlo methods to calculate the required integral. We will later extend the Monte Carlo class to use importance sampling with a given probability distribution function. First, we will declare a random sampling class which will take a probability distribution function and an inverse cumulative distribution function as arguments. The sampling class will use the transformation method to map values from a uniformly distributed region $[0, 1)$ to the required region over which the PDF is defined.

In [ ]:
# Define a fixed seed
np.random.seed(22092009)

class Sample():
    """Generates random variates given a PDF and corresponding inverse CDF.
    
    This class uses the transformation method to generate random deviates from a
    probability distribution given the corresponding inverse cumulative
    distribution function.
    
    Attributes
    ----------
    pdf : function
        The sampling probability distribution function.

    inverse_cdf : function
        The inverse cumulative distribution function of `pdf`.
    
    """
    
    def __init__(self, pdf, inverse_cdf):
        """Initialises Sample class.
        
        The class exposes a sample method to generate random deviates according
        to a probability distribution function.
        
        Parameters
        ----------
        pdf : function
            The normalised sampling probability distribution function.
        
        inverse_cdf : function
            The inverse cumulative distribution function of `pdf`.
        
        Raises
        ------
        ValueError
            If `pdf` or `inverse_cdf` is not a callable function.
        
        """
        
        if callable(pdf) == False or callable(inverse_cdf) == False:
            raise ValueError("Arguments `pdf` and `inverse_cdf` must be " +
                             "callable functions.")
        
        self.pdf = pdf
        self.inverse_cdf = inverse_cdf
    
    def sample(self):
        """Generates a random deviate from `pdf`.
        
        This method generates a random deviate using the transformation method.
        
        Returns
        -------
        rv : scalar
            A random variate drawn from `pdf`.
        
        Notes
        -----
        This function generates a sample from the `pdf` distribution using its
        corresponding `inverse_cdf` function as according to the transformation
        method. The inverse CDF function must return a numeric value for all
        values in the interval [0, 1).

        References
        ----------
        Numerical Recipes in C, Random Numbers, Transformation Method
        W. H. Press, Cambridge University Press

        Examples
        --------
        Sample a function from a uniform distribution on the interval [0, 2]

        >>> def pdf(x): return 1.0 / (2.0 - 0.0)
        >>> def inv_cdf(x): return x * 2
        >>> s = Sample(pdf, inv_cdf)
        >>> s.sample()
        1.7534370293465757

        """
        
        return self.inverse_cdf(np.random.rand())

We now define a generic Monte Carlo integration class. This will act as the parent class for the Monte Carlo integration method with importance sampling and adaptive recursion. We will use this class to hold information about the properties of the required integral as well as storing the results of the integration later.

In [ ]:
class MC_Integration():
    """Parent class to hold attributes for Monte Carlo integration methods.
    
    This class is used to hold all of the variables required to perform a Monte
    Carlo integration simulation. The class will provide argument validation and
    processing. The class exposes a method for simple Monte Carlo integration.
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the integral.

    b : scalar
        The upper limit of the integral.

    max_error : scalar
        The accuracy goal for the integration.
    
    max_samples : integer
        The maximum number of samples to perform the integral.
    
    plot : boolean
        A boolean to indicate if plot should be output.
    
    sample : Sample, optional, default = None
        The Sample instance used to generate random deviates. Defaults to a
        uniform distribution over the integration region.

    integral : scalar
        The value of the integral.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
        
    x_samples : list
        A list to hold the x samples which can be plotted later.
    
    """
    
    def __init__(self, fn, a, b, max_error, max_samples, plot, sample=None):
        """Initialises MC_Integration class.
        
        The class exposes an integration method which performs simple Monte
        Carlo until either the desired accuracy is achieved or the number of
        evaluations exceeds `max_samples`.
        
        Parameters
        ----------
        fn : function
            The integrand.

        a : scalar
            The lower limit of the integral.

        b : scalar
            The upper limit of the integral.

        max_error : scalar
            The accuracy goal for the integration.

        max_samples : integer
            The maximum number of samples to perform the integral.
        
        plot : boolean
            A boolean to indicate if plot should be output.
        
        sample : Sample, optional, default = None
            The Sample instance used to generate random deviates. Defaults to a
            uniform distribution over the integration region.
        
        Raises
        ------
        ValueError
            If `fn` is not a callable function.
            If `a`, `b` are not scalars or `a` is not less than `b`.
            If `max_error` is not a scalar.
            If `max_samples` is too small.
        
        """
        
        if callable(fn) == False:
            raise ValueError("The integrand `fn` must be a callable function.")
        
        if np.isfinite(a) == False or np.isfinite(b) == False:
            raise ValueError("Integral limits `a` and `b` must be finite " +
                             "numbers.")
        
        if b < a:
            raise ValueError("The upper limit `b` must be larger than the " +
                             "lower limit `a`.")
        
        if np.isfinite(max_error) == False:
            raise ValueError("Argument `max_error` must be a numeric input.")
        
        if max_samples < 5:
            raise ValueError("Argument `max_samples` must be an integer " +
                             "greater than 5.")
        
        self.fn = fn
        self.a = a
        self.b = b
        self.max_error = max_error
        self.max_samples = max_samples
        self.sample = sample
        self.plot = plot
        
        # If no sampling distribution provided
        if self.sample == None:
            
            # Define a uniform distribution on [a, b) using anyonymous functions
            pdf = lambda x : 1 / (b - a)
            inverse_cdf = lambda x : a + (b - a) * x
            
            # Create a sample instance for a uniform distribution
            self.sample = Sample(pdf, inverse_cdf)
        
        # Initiate variables
        self.integral = 0.0
        self.error = 1.0e30
        self.iterations = 0
        self.x_samples = []
    
    def integrate(self):
        """Performs the integration using a Monte Carlo simulation.
        
        This method performs a Monte Carlo simulation to integrate the function
        `fn` on [a, b). The attributes of the class are attributed to store the
        integral results and no value is returned.
        
        Raises
        ------
        ValueError
            If the integral did not converge to the desired accuracy in the
                given number of steps.
        
        See Also
        --------
        TrapeziumRule.integrate : performs integral using the trapezium rule.
        SimpsonsRule.integrate : performs integral using Simpsons rule.

        Notes
        -----
        Monte Carlo methods using random number sampling and the law of averages
        to approximate integrals. In lower dimensions, Newton-Cotes integrals
        are more efficient but Monte Carlo methods scale better with number of
        dimensions.

        References
        ----------
        Numerical Recipes in C, Random Numbers, Monte Carlo Integration

        """
        
        # Error validation performed on initiation, proceed to integral
        n = 0.0
        n_var = 0.0
        err = 1.0e30
        tot = 0
        mean_n = 1.0
        
        # Iterate until desired accuracy is achieved
        while (tot < 2500 or err > self.max_error * abs(mean_n)) and tot < self.max_samples:
            
            # Sample from distribution using Sample instance
            xn = self.sample.sample()
            self.x_samples.append(xn)
            
            # Compute the value of the integral based on xn
            yn = self.fn(xn) / self.sample.pdf(xn)
            n += yn
            n_var += yn ** 2
            tot += 1
            
            # Calculate the mean value of the integral based on samples
            mean_n = n / tot
            var = (n_var / tot - mean_n ** 2) / tot
            err = np.sqrt(var)
            
            # Print progress of integration
            if (tot % 250000 == 0):
                out = ("Steps: {:8d}, Integral: {:8.6f}, " +
                       "Error: {:8.3e}").format(tot, mean_n, err)
                print(out)
        
        # Update class attributes
        self.integral = mean_n
        self.error = err
        self.iterations = tot
        
        if self.plot:
            self.make_plot()
    
    def make_plot(self):
        # Internal function for graph plotting
        
        # Plot the abscissas
        plt.hist(self.x_samples, bins=24, edgecolor="black", color="#cccccc", density=True)

        # Plot sampling PDF
        X = np.linspace(self.a, self.b, 500)
        Y1 = [self.sample.pdf(x) for x in X]
        
        plt.plot(X, Y1, label="PDF", c="red")

        plt.xlabel("$x$")
        plt.ylabel("Proportion")
        plt.grid()
        plt.legend(loc=3)
        plt.show()

In [ ]:
class MonteCarlo_IS(MC_Integration):
    """Class to perform Monte Carlo integration with importance sampling.
    
    This class uses an iterative approach to estimate an integral using Monte
    Carlo sampling. We sample iteratively from a given probability distribution
    and compute the mean value of the integral. If no sampling distribution is
    provided, the method defaults to using a uniform PDF (i.e. not importance
    sampling).
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the integral.

    b : scalar
        The upper limit of the integral.

    max_error : scalar, optional, default = 1.0e-3
        The accuracy goal for the integration.
    
    plot : boolean, optional, default = True
        A boolean to indicate if plot should be output.
    
    sample : Sample instance, optional, default = None
            The Sample instance used to generate random deviates. Defaults to a
            uniform distribution over the integration region.

    integral : scalar
        The value of the integral.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
    
    """
    
    def __init__(self, fn, a, b, max_error=1.0e-3, plot=True, sample=None):
        """Initialises MonteCarlo_IS class.
        
        This class uses an iterative approach to estimate an integral using
        Monte Carlo sampling. We sample iteratively from a given probability
        distribution and compute the mean value of the integral. If no sampling
        distribution is provided, the method defaults to using a uniform PDF
        (i.e. not importance sampling).
        
        Parameters
        ----------
        fn : function
            The integrand.

        a : scalar
            The lower limit of the integral.

        b : scalar
            The upper limit of the integral.

        max_error : scalar, optional, default = 1.0e-3
            The accuracy goal for the integration.
        
        plot : boolean, optional, default = True

        sample : Sample instance, optional, default = None
                The Sample instance used to generate random deviates. Defaults
                to a uniform distribution over the integration region.
        
        See Also
        --------
        TrapeziumRule.integrate : performs integral using the trapezium rule.
        SimpsonsRule.integrate : performs integral using Simpsons rule.

        Notes
        -----
        Monte Carlo methods using random number sampling and the law of averages
        to approximate integrals. In lower dimensions, Newton-Cotes integrals
        are more efficient but Monte Carlo methods scale better with number of
        dimensions.

        References
        ----------
        Numerical Recipes in C, Random Numbers, Monte Carlo Integration

        Examples
        --------
        Integrate a function f on an interval x = 0.0 to x = 1.5708 with a
        uniform distribution

        >>> def f(x): return np.cos(x)
        >>> mc = MonteCarlo_IS(f, 0.0, 1.5708)
        >>> mc.integral
        0.9997844790190592

        Specify an accuracy goal of 1.0e-2 for the integral

        >>> mc = MonteCarlo_IS(f, 0.0, 1.5708, max_error=1.0e-2)
        >>> mc.integral
        1.0169522708504097

        Get error estimate and number of function evaluations

        >>> mc = MonteCarlo_IS(f, 0.0, 1.5708)
        >>> (mc.integral, mc.error, mc.iterations)
        (0.999297642053521, 0.0009999996266966391, 233824)

        Perform the integral using a linear distribution y = Ax + B
        >>> def pdf(x):
        >>>   A = -8 / np.pi ** 2
        >>>   B = 4 / np.pi
        >>>   return A * x + B
        >>> def inv_cdf(x):
        >>>   A = -8 / np.pi ** 2
        >>>   B = 4 / np.pi
        >>>   return -B / A - np.sqrt((B / A) ** 2 + 2 * x / A)
        >>> s = Sample(pdf, inv_cdf)
        >>> mc = MonteCarlo_IS(f, 0.0, 1.5708, sample=s)
        >>> (mc.integral, mc.error, mc.iterations)
        (1.0002155853673849, 0.000999994355192016, 16784)

        """
        
        # Inherit from MC_Integration class
        super().__init__(fn, a, b, max_error=max_error, max_samples=int(1.0e30),
                         sample=sample, plot=plot)
        
        # Compute the integral
        self.integrate()

#### **Simple Monte Carlo**
To integrate the quantum probability using a uniform distribution (i.e. not importance sampling), we do not pass any `Sample` obect.

In [ ]:
# Perform the integral using a flat distribution

def f(x):
    return np.exp(-x ** 2) / np.sqrt(np.pi)
    
mc = MonteCarlo_IS(f, 0.0, 2.0, max_error=1.0e-3, plot=True)
print("Integral: ", mc.integral)
print("Error: ", mc.error)
print("Function Evaluations: ", mc.iterations)

#### **Monte Carlo with Importance Sampling**
To use the Monte Carlo integration technique with importance sampling, we must provide a normalised PDF and the corresponding inverse CDF function. These two functions are provided to the `Sample` class as arguments and the resulting `Sample` instance is passed through the `sample` argument. In this example, we use the PDF suggested in the project guidelines.

In [ ]:
# Importance sampling, PDF and inverse CDF depend on the integration range
a = 0.0
b = 2.0

def pdf(x):
    global a, b
    if x < a or x > b:
        return 0
    else:
        # Define A and B, PDF normalised irrespective of integration range
        A = -0.48 / ((b - a) * (-0.24 * (b + a) + 0.98))
        B = 0.98 / ((b - a) * (-0.24 * (b + a) + 0.98))
        return A * x + B

def inverse_cdf(x):
    global a, b
    A = -0.48 / ((b - a) * (-0.24 * (b + a) + 0.98))
    B = 0.98 / ((b - a) * (-0.24 * (b + a) + 0.98))
    return -B / A - np.sqrt((B / A) ** 2 + (2 * (a * B + x) / A) + a ** 2)

def f(x):
    return np.exp(-x ** 2) / np.sqrt(np.pi)

s = Sample(pdf, inverse_cdf)
mc = MonteCarlo_IS(f, a, b, max_error=1.0e-3, sample=s, plot=True)
print("Integral: ", mc.integral)
print("Error: ", mc.error)
print("Function Evaluations: ", mc.iterations)

#### **Adaptive Monte Carlo**
We will define an adaptive Monte Carlo integration technique in which we use a recursive stratification scheme to split the integration region into smaller subregions based on their variance. The subregion with the largest variance is recursively bisected into smaller regions to reduce the total variance. Once the region has been appropriately stratified, each subregion undergoes simple Monte Carlo integration with a linear PDF. The final result is computed by taking an average over each of the subregions weighted by the width of each subregion, which will have a smaller variance than the variance of the single region scheme. To achieve a given accuracy, the same integral is repeated over the subregions multiple times until the variance of the means is less than the required value.

To implement this recursive stratification scheme, we will need to define a `Subregion` class which contains all the information necessary to perform simple Monte Carlo integration on itself.

In [ ]:
class Subregion(MC_Integration):
    """Class to perform Monte Carlo integration on a subregion.
    
    This class allows each subregion instance to hold all of the necessary
    information as attributes to perform simple Monte Carlo integration on
    itself.
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the subregion.

    b : scalar
        The upper limit of the subregion.
    
    sample : Sample instance
            The Sample instance used to generate random deviates for importance
            sampling.
    
    use_IS : boolean
            Boolean to indicate if importance sampling should be used on the
            subregion. If True, the class applies the given PDF
            (y = -0.48x + 0.98), appropriately normalised, to each subregion.

    max_samples : integer, optional, default = int(1.0e30)
            The maximum number of steps to perform the integral.

    integral : scalar
        The value of the integral in the subregion.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
    
    """
    
    def __init__(self, fn, a, b, use_IS, max_samples=100):
        """Initialises Subregion class.
        
        This class allows each subregion instance to hold all of the necessary
        information as attributes to perform simple Monte Carlo integration on
        itself.
        
        Parameters
        ----------
        fn : function
            The integrand.
        
        a : scalar
            The lower limit of the subregion.
        
        b : scalar
            The upper limit of the subregion.
        
        use_IS : boolean
            Boolean to indicate if importance sampling should be used on the
            subregion.
        
        max_samples : integer, optional, default = 100
            The maximum number of steps to perform the integral.

        """
        
        self.use_IS = use_IS
        
        # We sample to max_samples, set max_error = 1.0e-30
        max_error = 1.0e-30
        
        if use_IS == True:
            # Define A and B, PDF normalised irrespective of integration range
            # A = -0.48 / ((b - a) * (-0.24 * (b + a) + 0.98))
            # B = 0.98 / ((b - a) * (-0.24 * (b + a) + 0.98))

            # Define adapted PDF over subregion
            Ap = (fn(b) - fn(a)) / (b - a)
            k_num = -2.0 * (fn(a) - fn(b))
            k_den = (fn(a) + fn(b)) * Ap * (b - a) ** 2
            k = k_num / k_den
            A = k * Ap
            B = 1 / (b - a) - 0.5 * k * Ap * (a + b)


            # Define normalised PDF and inverse CDF for subregion
            def pdf(x):
                if x < a or x > b:
                    return 0
                else:
                    return A * x + B

            def inverse_cdf(x):
                return -B / A + np.sign(A) * np.sqrt((B / A) ** 2 + (2 * (a * B + x) / A) + a ** 2)
            
            sample = Sample(pdf, inverse_cdf)
        else:
            sample = None

        # Inherit from MC_Integrate class
        super().__init__(fn, a, b, max_error=max_error, sample=sample,
                         max_samples=max_samples, plot=False)
        
        # Compute the integral on the subregion
        self.integrate()
    
    def bisect(self):
        """Bisects a subregion.
        
        This method bisects a subregion into two equal-width subregions.
        
        Returns
        -------
        subregions : list
            Contains two Subregion instances for the bisected subregion.
        
        Examples
        --------
        Define a subregion on an interval and bisect the subregion
        
        >>> s = Subregion(fn, 0.0, 1.5708)
        >>> srs = s.bisect()
        >>> (srs[0].a, srs[0].b, srs[1].a, srs[1].b)
        (0.0, 0.7854, 0.7854, 1.5708)

        """
        
        midpoint = self.midpoint()
        return [
            Subregion(self.fn, self.a, midpoint, use_IS=self.use_IS),
            Subregion(self.fn, midpoint, self.b, use_IS=self.use_IS)
        ]
    
    def width(self):
        # Return width of subregion
        return self.b - self.a
    
    def midpoint(self):
        # Return midpoint of subregion
        return self.a + 0.5 * (self.b - self.a)

In [ ]:
class MonteCarlo_AS(MC_Integration):
    """Class to perform Monte Carlo integration using adaptive sampling.
    
    This class performs a Monte Carlo integration using recursive stratified
    sampling (MISER algorithm) as an adaptive scheme. The region with the
    largest variance is recursively bisected until the total error is within the
    tolerance.
    
    Attributes
    ----------
    fn : function
        The integrand.

    a : scalar
        The lower limit of the subregion.

    b : scalar
        The upper limit of the subregion.
        
    sample : Sample instance, optional, default = None
            The Sample instance used to generate random deviates. Defaults to a
            uniform distribution over the integration region.

    max_error : scalar, optional, default = 1.0e-3
            The accuracy goal for the integration.
    
    use_IS : boolean, optional, default = False
            Boolean to indicate if importance sampling should be used on the
            subregion. If True, the class applies the given PDF
            (y = -0.48x + 0.98), appropriately normalised, to each subregion.

    integral : scalar
        The value of the integral in the subregion.

    error : scalar
        An estimate for the error on the integral.

    iterations : integer
        The number of function evaluations required.
    
    subregions : list
        A list holding the subregions of the integral
    
    """
    
    def __init__(self, fn, a, b, max_error=1.0e-3, use_IS=False, plot=False):
        """Initialises MonteCarlo_AS class.
        
        This class performs a Monte Carlo integration using recursive stratified
        sampling (1D MISER algorithm) as an adaptive scheme. The region with the
        largest variance is recursively bisected until the total error is within
        the tolerance or the number of subregions exceeds maximum subregions
        limit.
        
        Parameters
        ----------
        fn : function
            The integrand.

        a : scalar
            The lower limit of the integral.

        b : scalar
            The upper limit of the integral.

        max_error : scalar, optional, default = 1.0e-3
            The accuracy goal for the integration.

        use_IS : boolean, optional, default = False
            Boolean to indicate if importance sampling should be used on the
            subregion. If True, the class applies the given PDF
            (y = -0.48x + 0.98), appropriately normalised, to each subregion.
        
        """
        
        # Inherit from MC_Integration class
        max_samples = int(1.0e30)
        self.use_IS = use_IS
        super().__init__(fn, a, b, max_error=max_error, plot=plot,
                         max_samples=max_samples)
        
        # Compute the integral
        self.integrate()
    
    def integrate(self):
        """Performs the integration using adaptive Monte Carlo.
        
        This method performs a Monte Carlo simulation to integrate the function
        `fn` over the region `a` to `b`.
        
        See Also
        --------
        TrapeziumRule.integrate : performs integral using the trapezium rule.
        SimpsonsRule.integrate : performs integral using Simpsons rule.
        MC_Integration.integrate : performs integral using Monte Carlo methods.

        Notes
        -----
        Monte Carlo methods using random number sampling and the law of averages
        to approximate integrals. In the adaptive sampling 1D MISER scheme, the
        integration subregion with the largest is bisected recursively at each
        step and the combined variance is less than the variance of the initial
        subregion.

        References
        ----------
        Numerical Recipes in C, Random Numbers, Adaptive Monte Carlo Integration

        Examples
        --------
        Integrate a function f on [0.0, 1.5708) using MISER with flat PDF

        >>> def f(x): return np.cos(x)
        >>> mc = MonteCarlo_AS(f, 0.0, 1.5708)
        >>> mc.integral
        1.0006260528995337

        Specify an accuracy goal of 1.0e-5 for the integral

        >>> mc = MonteCarlo_AS(f, 0.0, 1.5708, max_error=1.0e-5)
        >>> mc.integral
        0.9999878990120468

        Get information about the integral

        >>> mc = MonteCarlo_AS(f, 0.0, 1.5708)
        >>> (mc.integral, mc.error, mc.iterations, mc.srs)
        (1.0011003084835095, 0.0009787300215640482, 2400, 13)

        """
        
        # Declare the initial bisected region
        max_subregions = 2000
        max_samples = 5
        mid = self.a + 0.5 * (self.b - self.a)
        self.subregions = [
            Subregion(self.fn, self.a, mid, use_IS = self.use_IS, max_samples=max_samples),
            Subregion(self.fn, mid, self.b, use_IS=self.use_IS, max_samples=max_samples)
        ]
        
        # Declare variables for total error and total integral
        grand_error = 1.0e30
        grand_integral = 1.0
        
        # Creating the initial subregions requires 2 * max_samples iterations 
        iters = 2 * max_samples
        
        while grand_error > self.max_error * np.abs(grand_integral) and len(self.subregions) < max_subregions:
            # Find subregion with highest variance (i.e. error)
            j = 0
            for i in range(len(self.subregions)):
                if self.subregions[i].error > self.subregions[j].error:
                    j = i

            # Bisect subregion with largest error
            bisected_regions = self.subregions[j].bisect()
            
            # Replace current subregion, append the other subregion
            self.subregions[j] = bisected_regions[0]
            self.subregions.append(bisected_regions[1])
            
            # Bisection of subregion requires 2 * max_samples iterations
            iters += 2 * max_samples
            
            # Compute new value for total integral and error
            grand_error = np.sqrt(sum([region.error ** 2 for region in self.subregions]))
            grand_integral = sum([region.integral for region in self.subregions])
        
        # Either integral converged or maximum subregion limit exceeded
        self.integral = grand_integral
        self.error = grand_error
        self.iterations = iters
        self.srs = len(self.subregions)

        if self.plot == True:
            self.make_plot()
    
    def make_plot(self):
        # Internal function for graph plotting

        # Get midpoints and widths of subregions
        X = np.array([[reg.midpoint(), reg.width()] for reg in self.subregions])
        
        # Plot histogram of subregion widths
        plt.hist(X[:, 1], edgecolor="black")
        plt.xlabel("Subregion Width")
        plt.ylabel("Population")
        plt.grid()
        plt.show()
        
        # Plot midpoints and widths
        plt.plot(X[:, 0], X[:, 1], "x", color="black")
        plt.xlabel("$x$")
        plt.ylabel("Subregion Width")
        plt.grid()
        plt.show()

In [ ]:
# Without importance sampling

def f(x):
    return np.exp(-x ** 2) / np.sqrt(np.pi)

mc = MonteCarlo_AS(f, 0.0, 2.0, max_error=1.0e-3, use_IS=False, plot=True)
print("Integral: ", mc.integral)
print("Error: ", mc.error)
print("Function Evaluations: ", mc.iterations)
print("Subregions: ", mc.srs)

In [ ]:
# With importance sampling

def f(x):
    return np.exp(-x ** 2) / np.sqrt(np.pi)

mc = MonteCarlo_AS(f, 0.0, 2.0, max_error=1.0e-3, use_IS=True, plot=True)
print("Integral: ", mc.integral)
print("Error: ", mc.error)
print("Function Evaluations: ", mc.iterations)
print("Subregions: ", mc.srs)

#### **Miscellaneous Plots**
The values for these plots were acquired by manually updating the integration parameters and running several trials.

In [ ]:
x1 = [3, 4, 5, 6, 7, 8, 9]
y1 = [17, 33, 129, 513, 1025, 4097, 16385]  # Trapezium Rule
y2 = [9, 17, 33, 65, 65, 129, 257]  # Simpson's Rule

plt.semilogy(x1, y1, marker="x", label="Trapezium Rule", c="black")
plt.semilogy(x1, y2, marker="x", label="Simpson's Rule", c="red")
plt.xlabel("Accuracy, $-\log_{10}(\epsilon)$")
plt.ylabel("Evaluations, $K$")
plt.legend()
plt.grid()
plt.show()

In [ ]:
x1 = [1, 2, 3, 4]
y1 = [78, 6091, 610119, 61075761]  # Uniform Distribution
y2 = [5, 726, 73031, 7341744]  # Linear Distribution

x2 = [4, 5, 6, 7, 8]
y3 = [61075761, 5.4e9, 5.0e11, 4.7e13, 4.3e15]  # Uniform distribution
y4 = [7341744, 8.9e8, 1.0e11, 1.1e13, 1.3e15]  # Linear distribution

plt.semilogy(x1, y1, marker="x", label="Uniform Distribution", c="black")
plt.semilogy(x1, y2, marker="x", label="Linear Distribution", c="red")
plt.semilogy(x2, y3, marker="x", c="black", ls="dashed")
plt.semilogy(x2, y4, marker="x", c="red", ls="dashed")
plt.xlabel("Accuracy, $-\log_{10}(\epsilon)$")
plt.ylabel("Evaluations, $K$")
plt.legend()
plt.grid()
plt.show()

In [ ]:
xis = [1, 2, 3, 4]
yis1 = [78, 6091, 610119, 61075761]
yis2 = [5, 726, 73031, 7341744]

xrss = [1, 2, 3, 4, 5, 6, 7]
yrss = [20, 30, 160, 780, 3500, 15740, 74480]

xaisrss = [1, 2, 3, 4, 5, 6, 7, 8, 9]
yaisrss = [20, 20, 40, 100, 250, 620, 1580, 4050, 9800]

xtrsr = [3, 4, 5, 6, 7, 8, 9]
ytr = [17, 33, 129, 513, 1025, 4097, 16385]  # Trapezium Rule
ysr = [9, 17, 33, 65, 65, 129, 257]  # Simpson's Rule

plt.semilogy(xis, yis1, marker="x", label="MC Uniform", c="black")
plt.semilogy(xis, yis2, marker="x", label="MC IS", c="red")
plt.semilogy(xrss, yrss, marker="x", label="MC RSS", c="orange")
plt.semilogy(xaisrss, yaisrss, marker="x", label="MC AIS+RSS", c="green")
plt.semilogy(xtrsr, ytr, marker="x", label="Trapezium", c="blue")
plt.semilogy(xtrsr, ysr, marker="x", label="Simpson's", c="purple")
plt.xlabel("Accuracy, $-\log_{10}(\epsilon)$")
plt.ylabel("Evaluations, $K$")
plt.legend()
plt.grid()
plt.show()